In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras import layers

In [ ]:
import PIL.Image as Image
import cv2
import os

In [ ]:
data_source = 'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz'
flowers_data = tf.keras.utils.get_file('flower_photos',origin=data_source,cache_dir='.',untar=True)

In [ ]:
import pathlib

In [ ]:
flowers_data = pathlib.Path(flowers_data)

In [ ]:
len(list(flowers_data.glob('*/*.jpg')))

3670

In [ ]:
flower_images_dict = {
    'dandelion': list(flowers_data.glob('dandelion/*')),
    'roses': list(flowers_data.glob('roses/*')),
    'tulips': list(flowers_data.glob('tulips/*')),
    'sunflowers': list(flowers_data.glob('sunflowers/*')),
    'daisy': list(flowers_data.glob('daisy/*')),
}

In [ ]:
flower_dict_labels = {
    'dandelion':0,
    'roses':1,
    'tulips':2,
    'sunflowers':3,
    'daisy':4
}

In [ ]:
x,y = [],[]

for flower_name,images in flower_images_dict.items():
  for image in images:
    img = cv2.imread(str(image))
    resized_image = cv2.resize(img,(224,224))
    x.append(resized_image)
    y.append(flower_dict_labels[flower_name])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x = np.array(x)
y = np.array(y) 

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
x_train_scaled = np.divide(x_train,255)
x_test_scaled = np.divide(x_test,255)

In [ ]:
class VGGNetLayer1(layers.Layer):

    def __init__(self,channels,kernel_size,padding,pool_size,pool_stride,activation='relu',input_shape=''):
        super().__init__()
        self.conv1 = layers.Conv2D(filters=channels,kernel_size=kernel_size,padding=padding,activation=activation,input_shape=input_shape)
        self.conv2 = layers.Conv2D(filters=channels,kernel_size=kernel_size,padding=padding,activation=activation)
        self.pool1 = layers.MaxPooling2D(pool_size=pool_size,strides=pool_stride)

    def call(self,input_tensor):
        x = self.conv1(input_tensor)
        x = self.conv2(x)
        x = self.pool1(x)
        return tf.nn.relu(x)

class VGGNetLayer2(layers.Layer):

    def __init__(self,channels,kernel_size,padding,pool_size,pool_stride,activation='relu',input_shape=''):
        super().__init__()
        self.block1 = layers.Conv2D(filters=channels,kernel_size=kernel_size,padding=padding,activation=activation,input_shape=input_shape)
        self.block2 = layers.Conv2D(filters=channels,kernel_size=kernel_size,padding=padding,activation=activation)
        self.block3 = layers.Conv2D(filters=channels,kernel_size=kernel_size,padding=padding,activation=activation)
        self.pool2 = layers.MaxPooling2D(pool_size=pool_size,strides=pool_stride)
    
    def call(self,input_tensor):
        x = self.block1(input_tensor)
        x = self.block2(x)
        x = self.block3(x)
        x = self.pool2(x)
        return tf.nn.relu(x)


class VGGNet_16(keras.Model):

    def __init__(self,num_classes=1000):
        super().__init__()
        self.layer1 = VGGNetLayer1(input_shape=(224,224,3),channels=64,kernel_size=(3,3),padding='same',activation='relu',pool_size=(2,2),pool_stride=(2,2))
        self.layer2 = VGGNetLayer1(input_shape=(),channels=128,kernel_size=(3,3),padding='same',activation='relu',pool_size=(2,2),pool_stride=(2,2))
        self.layer3 = VGGNetLayer2(input_shape=(),channels=256,kernel_size=(3,3),padding='same',activation='relu',pool_size=(2,2),pool_stride=(2,2))
        self.layer4 = VGGNetLayer2(input_shape=(),channels=512,kernel_size=(3,3),padding='same',activation='relu',pool_size=(2,2),pool_stride=(2,2))
        self.layer5 = VGGNetLayer2(input_shape=(),channels=512,kernel_size=(3,3),padding='same',activation='relu',pool_size=(2,2),pool_stride=(2,2))
        self.flatten = layers.Flatten()
        self.fully_connected1 = layers.Dense(4096,activation='relu')
        self.fully_connected2 = layers.Dense(4096,activation='relu')
        self.classfier = layers.Dense(num_classes,activation='softmax')

    def call(self,input_tensor):
        x = self.layer1(input_tensor)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)
        x = self.flatten(x)
        x = self.fully_connected1(x)
        x = self.fully_connected2(x)
        return self.classfier(x)

In [ ]:
model = VGGNet_16(num_classes=5)

In [ ]:
model.compile(
    optimizer='adam',
    metrics =['accuracy'],
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [ ]:
model.fit(x_train_scaled,y_train,epochs=1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


92/92 [==============================] - 60s 477ms/step - loss: 1.6154 - accuracy: 0.2302


In [ ]:
model.summary()

Model: "vgg_net_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg_net_layer1 (VGGNetLayer  multiple                 38720     
 1)                                                              
                                                                 
 vgg_net_layer1_1 (VGGNetLay  multiple                 221440    
 er1)                                                            
                                                                 
 vgg_net_layer2 (VGGNetLayer  multiple                 1475328   
 2)                                                              
                                                                 
 vgg_net_layer2_1 (VGGNetLay  multiple                 5899776   
 er2)                                                            
                                                                 
 vgg_net_layer2_2 (VGGNetLay  multiple                 7